In [1]:
from astropy.io import fits
import numpy as np
import pandas as pd

In [2]:
haloid = 166
instrument = "athena_wfi"
hdul_bkg = fits.open(f"halo_{haloid}_bkg_evt_full_{instrument}.fits")
data_bkg = hdul_bkg['EVENTS'].data
header = hdul_bkg['EVENTS'].header

x = data_bkg['X']
y = data_bkg['Y']
energy = data_bkg["Energy"] * 1e-3  # Assuming originally in eV, convert to keV

# Center of the image
x0 = header['TCRPX2']
y0 = header['TCRPX3']

# Pixel scale in arcmin
pixscale_deg = abs(header['TCDLT2'])
pixscale_arcmin = pixscale_deg * 60.0

# Radial distances in arcmin
radii = np.sqrt((x - x0)**2 + (y - y0)**2) * pixscale_arcmin

In [3]:
# Annular bin edges in arcmin
edges = np.array([0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 1., 1.3333337, 2.0000003, 2.6666663, 4.3333333, 6.0000003, 7.6666663])
trarcmin = (edges[:-1] + edges[1:]) / 2
trwidth = (edges[1:] - edges[:-1]) / 2

# Energy bands in keV
energy_bands = [
    (0.7, 1.0),
    (1.0, 1.3),
    (1.3, 1.6),
    (1.6, 2.0),
    (2.0, 2.7),
    (2.7, 3.4),
    (3.4, 3.8),
    (3.8, 4.3),
    (4.3, 5.0),
    (5.0, 7.0)]

In [4]:
hard_band = (energy >= 9.0) & (energy < 13.0)
radii_hard = radii[hard_band]

exposure = 1e5  # seconds

hard_count_rate = []

for i in range(len(edges) - 1):
    in_annulus_hard = (radii_hard >= edges[i]) & (radii_hard < edges[i + 1])
    count_hard = np.sum(in_annulus_hard)
    area_hard = np.pi * (edges[i + 1]**2 - edges[i]**2)  # arcmin^2
    rate_hard = count_hard / (area_hard * exposure)  # counts/sec/arcmin^2
    hard_count_rate.append(rate_hard)

hard_count_rate_new = [0.001 if item == 0 else item for item in hard_count_rate]

In [ ]:
# --- STEP 2: Loop over bands
for idx, (emin, emax) in enumerate(energy_bands, start=1):
    in_band = (energy >= emin) & (energy < emax)
    band_radii = radii[in_band]

    bgcountrate = []
    rate_list = []

    for i in range(len(edges) - 1):
        rmin, rmax = edges[i], edges[i + 1]
        in_annulus = (band_radii >= rmin) & (band_radii < rmax)
        count = np.sum(in_annulus)
        area = np.pi * (rmax**2 - rmin**2)  # arcmin^2
        exposure = 1e5
        rate = count / (area * exposure)  # counts/sec/arcmin^2
        rate_list.append(rate)
        count_rate_norm = rate / hard_count_rate_new[i]  # normalized to hard band
        bgcountrate.append(count_rate_norm)
    print("rate: ",rate_list)
    print("hard: ", hard_count_rate)
    print("hard new: ", hard_count_rate_new)
    print(bgcountrate)

    df = pd.DataFrame({
    'rrarcmin': trarcmin,
    'zero1': np.zeros_like(trarcmin),
    'zero2': np.zeros_like(trarcmin),
    'zero3': np.zeros_like(trarcmin),
    'bgcountrate': rate_list
    #'bgcountrate': np.nan_to_num(bgcountrate, nan=0.0, posinf=0.0)
    })

    # Write to file
    header_lines = "# rrarcmin       zero       zero       zero  bgcountrate\n#\n"
    outname = f"fgbg/bg_profnew_{int(emin*1000):04d}_{int(emax*1000):04d}.dat"

    with open(outname, "w") as f:
        f.write(header_lines)
        df.to_string(f, index=False, header=False, float_format="%.6e")

print("Done generating files for all bands.")

rate:  [0.0012732395447351626, 0.0, 0.0002546479089470326, 0.00018189136353359458, 0.00042441318157838775, 0.00034724714856413535, 9.094568176679729e-05, 0.00014147106052612925, 0.00019947419534184217, 0.000155516920314211, 7.591690026314559e-05, 9.20825961869428e-05, 0.00010968047654211937, 0.00012845341216081662, 0.0001479908660264337]
hard:  [0.002546479089470325, 0.0012732395447351626, 0.0028011269984173585, 0.0029102618165375133, 0.002263536968418068, 0.0016204866932992982, 0.00268289761212052, 0.002864788975654117, 0.002567699748549245, 0.001817092437355518, 0.001207508432487391, 0.0010374639170395556, 0.0014127500187440152, 0.0018367913813729432, 0.0020268739573629786]
hard new:  [0.002546479089470325, 0.0012732395447351626, 0.0028011269984173585, 0.0029102618165375133, 0.002263536968418068, 0.0016204866932992982, 0.00268289761212052, 0.002864788975654117, 0.002567699748549245, 0.001817092437355518, 0.001207508432487391, 0.0010374639170395556, 0.0014127500187440152, 0.0018367913